In [342]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import re
from sklearn.feature_extraction.text import TfidfVectorizer


In [343]:
%matplotlib inline

In [344]:
datapath = os.path.expanduser('~') + '/meddata/'
dfVar = pd.read_csv(datapath + 'training_variants') #,index_col='ID')

In [345]:
dfVar.head()

,ID,Gene,Variation,Class
0,0,FAM58A,Truncating Mutations,1
1,1,CBL,W802*,2
2,2,CBL,Q249E,2
3,3,CBL,N454D,3
4,4,CBL,L399V,4


In [346]:
file = open(datapath + 'training_text', 'r', encoding = 'utf-8') 
rawText = file.read()

In [347]:
lines = rawText.split('\n')[1:]

splitted = [line.split('||') for line in lines]

dfText = pd.DataFrame(splitted, columns=['ID','Text'])

dfText['ln'] = dfText['ID'].map(lambda x: len(x))
dfText = dfText[dfText['ln'] > 0]
dfText.drop('ln',axis=1,inplace=True)
dfText['ID'] = dfText['ID'].map(lambda n:int(n))


In [348]:
dfText.head()

,ID,Text
0,0,Cyclin-dependent kinases (CDKs) regulate a var...
1,1,Abstract Background Non-small cell lung canc...
2,2,Abstract Background Non-small cell lung canc...
3,3,Recent evidence has demonstrated that acquired...
4,4,Oncogenic mutations in the monomeric Casitas B...


In [349]:
df = pd.merge(dfText, dfVar, on='ID')
df.set_index('ID',drop=True,inplace=True)

In [350]:
df['hasAbstract'] = df['Text'].map(lambda txt: 
                                   'abstract' in txt.lower())


In [351]:
df.head()

,Text,Gene,Variation,Class,hasAbstract
ID,,,,,
0,Cyclin-dependent kinases (CDKs) regulate a var...,FAM58A,Truncating Mutations,1,False
1,Abstract Background Non-small cell lung canc...,CBL,W802*,2,True
2,Abstract Background Non-small cell lung canc...,CBL,Q249E,2,True
3,Recent evidence has demonstrated that acquired...,CBL,N454D,3,False
4,Oncogenic mutations in the monomeric Casitas B...,CBL,L399V,4,False


In [352]:
df['hasIntroduction'] = df['Text'].map(lambda txt: 
                                   'introduction' in txt.lower())
    

In [353]:
abstracts = []
for index in range(0, df.shape[0]):
    abstracts.append(df.ix[index,'Text'].partition('Introduction')[0])


df['Abstracts'] = pd.Series (abstracts, index=df.index-1)

3321


In [354]:
df.head()

,Text,Gene,Variation,Class,hasAbstract,hasIntroduction,Abstracts
ID,,,,,,,
0,Cyclin-dependent kinases (CDKs) regulate a var...,FAM58A,Truncating Mutations,1,False,False,Abstract Background Non-small cell lung canc...
1,Abstract Background Non-small cell lung canc...,CBL,W802*,2,True,True,Abstract Background Non-small cell lung canc...
2,Abstract Background Non-small cell lung canc...,CBL,Q249E,2,True,True,Recent evidence has demonstrated that acquired...
3,Recent evidence has demonstrated that acquired...,CBL,N454D,3,False,True,Oncogenic mutations in the monomeric Casitas B...
4,Oncogenic mutations in the monomeric Casitas B...,CBL,L399V,4,False,True,Oncogenic mutations in the monomeric Casitas B...
